In [1]:
import sys
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt



path='/home/jbohne/sbu/'
sys.path.append(f'{path}taq_db/TAQ-Query-Scripts/data')
sys.path.append(f'{path}research/Adaptive_Signal_Estimation_Private')

from trend_filtering.adaptive_tf import adaptive_tf
from trend_filtering.test_adaptive_tf import test_adaptive_tf
from profiling.profiler import profile

In [2]:
trades=pd.read_csv(f'{path}taq_db/TAQ-Query-Scripts/data/AAPL_trades.csv',index_col=0).dropna()

price=trades['Trade_Price'].values

In [3]:
price.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

### Cross Validation is 10 fold + OOS Fit

In [4]:


profile(test_adaptive_tf,None)



Duality Gap is 47.79588240651872
Duality Gap is 47.79586870709745
Duality Gap is 47.8092622501726
Duality Gap is 47.80919849270982
Duality Gap is 47.80919130905736
Duality Gap is 47.8091912392883
Duality Gap is 47.809191238592014
Duality Gap is 47.80919123858506
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
Duality Gap is 47.80919123858499
         8675 function calls (8495 primitive calls) in 0.325 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  per